In [1]:
%run setup_levy_model_generic.py

2024-06-18 09:48:04.518596: I tensorflow/core/util/port.cc:113] oneDNN custom operations are on. You may see slightly different numerical results due to floating-point round-off errors from different computation orders. To turn them off, set the environment variable `TF_ENABLE_ONEDNN_OPTS=0`.
2024-06-18 09:48:04.554486: I external/local_tsl/tsl/cuda/cudart_stub.cc:31] Could not find cuda drivers on your machine, GPU will not be used.
2024-06-18 09:48:04.742721: E external/local_xla/xla/stream_executor/cuda/cuda_dnn.cc:9261] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
2024-06-18 09:48:04.742756: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:607] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
2024-06-18 09:48:04.770857: E external/local_xla/xla/stream_executor/cuda/cuda_blas.cc:1515] Unable to register cuBLAS factory: Attempting to

2.15.1


INFO:root:Performing 2 pilot runs with the lfm model...
INFO:root:Shape of parameter batch after 2 pilot simulations: (batch_size = 2, 5)
INFO:root:Shape of simulation batch after 2 pilot simulations: (batch_size = 2, 1000, 2)
INFO:root:No optional prior non-batchable context provided.
INFO:root:No optional prior batchable context provided.
INFO:root:No optional simulation non-batchable context provided.
INFO:root:No optional simulation batchable context provided.


[[2.6 3.  0.4 0.2 1.3]] [[1.1 3.  0.3 0.3 0.4]]


INFO:root:Loaded loss history from checkpoints//generic/history_150.pkl.
INFO:root:Networks loaded from checkpoints//generic/ckpt-150
INFO:root:Performing a consistency check with provided components...
INFO:root:Done.


In [2]:
# Loading data
import pandas as pd
import os
import fnmatch
import glob 
from keras.utils import to_categorical

In [3]:
def config_observed_data(forward_dict):
    """Configure the output of the GenerativeModel for a BayesFlow setup."""

    # Prepare placeholder dict
    out_dict = {}

    # Extract simulated response times
    data = forward_dict["data"]
    data = data[np.newaxis,:,:]
    out_dict["summary_conditions"] = data.astype(np.float32)

    # Make inference network aware of varying numbers of trials
    # We create a vector of shape (batch_size, 1) by repeating the sqrt(num_obs)
    vec_num_obs = np.sqrt(data.shape[0]) * np.ones((data.shape[0], 1))
    out_dict["direct_conditions"] = np.sqrt(vec_num_obs).astype(np.float32)

    return out_dict

In [4]:
def get_posterior_means(file):
    raw_data = pd.read_csv(file)
    # raw_data["stim"] = raw_data["stim"]-1
    # raw_data["rt"] = raw_data["rt"]/1000
    raw_data["rt"] = (raw_data["resp"]*2 - 1) * raw_data["rt"] # to negate wrong rts
    condition = raw_data["resp"]*0 # set it all to 0
    data = np.c_[condition, raw_data["rt"].to_numpy()]
    input_dict = {
        'data': data
    }
    
    theta_hat = amortizer.sample(config_observed_data(input_dict), 5000, to_numpy=True)
    std_means = np.mean(theta_hat, axis=0)
    return(prior_means + std_means*prior_stds)

In [8]:
# Loop through each datafile and estimate 
path_to_data = 'data/cognitive_control/'
PAR = pd.DataFrame(columns=['task', 'sub', 'condition', 'a', 'v', 't', 'st', 'alpha'])
tasks = ['flanker', 'stroop', 'simon']
conditions = ['congruent', 'incongruent']

for task in tasks:        
    for condition in conditions:
        mask = os.path.join(path_to_data, "raw_data/", task + '_' + condition + '_subject_*.csv')
        print(mask)
        datafiles = glob.glob(mask) 

        for file in datafiles:
            print(file)
            par = get_posterior_means(file)
            print(np.round(par,2))
            result = pd.DataFrame(par.reshape(1,5),columns=['a', 'v', 't', 'st', 'alpha'])
            result['task'] = task
            result['sub'] = file.split('_')[-1].split('.')[0]
            result['condition'] = condition
            PAR = pd.concat([PAR, result], ignore_index=True)    

data/cognitive_control/raw_data/flanker_congruent_subject_*.csv
data/cognitive_control/raw_data/flanker_congruent_subject_1.csv
[[2.09 5.86 0.23 0.09 1.47]]
data/cognitive_control/raw_data/flanker_congruent_subject_10.csv


/tmp/ipykernel_6788/2977786649.py:21: FutureWarning: The behavior of DataFrame concatenation with empty or all-NA entries is deprecated. In a future version, this will no longer exclude empty or all-NA columns when determining the result dtypes. To retain the old behavior, exclude the relevant entries before the concat operation.
  PAR = pd.concat([PAR, result], ignore_index=True)


[[1.45 4.1  0.28 0.11 1.28]]
data/cognitive_control/raw_data/flanker_congruent_subject_11.csv
[[1.48 3.91 0.23 0.09 1.3 ]]
data/cognitive_control/raw_data/flanker_congruent_subject_12.csv
[[2.13 5.74 0.25 0.09 1.49]]
data/cognitive_control/raw_data/flanker_congruent_subject_13.csv
[[2.   5.24 0.22 0.1  1.44]]
data/cognitive_control/raw_data/flanker_congruent_subject_14.csv
[[1.58 4.36 0.25 0.1  1.32]]
data/cognitive_control/raw_data/flanker_congruent_subject_15.csv
[[1.74 4.93 0.22 0.09 1.35]]
data/cognitive_control/raw_data/flanker_congruent_subject_16.csv
[[1.54 4.41 0.25 0.1  1.34]]
data/cognitive_control/raw_data/flanker_congruent_subject_17.csv
[[2.1  5.61 0.22 0.09 1.42]]
data/cognitive_control/raw_data/flanker_congruent_subject_18.csv
[[1.99 5.11 0.22 0.1  1.4 ]]
data/cognitive_control/raw_data/flanker_congruent_subject_19.csv
[[2.02 5.23 0.24 0.09 1.39]]
data/cognitive_control/raw_data/flanker_congruent_subject_1934.csv
[[2.88 3.16 0.28 0.16 1.5 ]]
data/cognitive_control/raw_da

In [9]:
PAR.to_csv(path_to_data + 'results/levy_mat_parameters.csv', index=False)